[08/03/2022]
1. Arm Emulation
    - Emulation : A software that can imitate different architecture.
    - QUMU, [x86 emulation in browser](https://bellard.org/jslinux/)
    - objdump -D \<filename>.o (Looking at the instruction value), eg. 0xe080 0001 (1110 0000 1000 0000 ... 0001)
    - <img src="images/dataproc1.png" align="center" width="50%" height="50%"/>
    - <img src="images/instrctn3.png" align="center" width="50%" height="50%"/>
    - `DP bits` to understand which instruction it is!(Generally set of constraint bits), **For understanding which instruction it is move from more constraint to less contraint instruction**
    - gcc -o \<exec> \<cpgm>.c \<assembly>.s (To link assembly to C while executing), **ref analyze.c**
    
2. Arm state:
    - <img src="images/armstate.png" align="center" width="50%" height="50%"/>
    - Declaring a struct to handle the state. **ref armemu.c**, armemu_init() -> int armemu() -> armemu_one()
   

[10/03/2022]
1. Arm Emulation:
    * Where does arm state lives in the mem
    * <img src="images/armstate1.png" align="center" width="50%" height="50%"/>
    * **Thus we should have anything in the code that directly reference stack memory in the arm state.**
2. Data Processing:
    * Implementing `sub`, `add`, `cmp`, (later : AND, ORR)
    * `mul`
        * Not using 'Rn' as present in the instruction set.
    * Single Data Transfer:
        * ldr, str, ldrb, strb
        * Understanding **ldr r0, [r1]**:
            * 0xe5910000 -> 0b 1110 0101 1001 0001 0000 0000 0000 0000
            * <img src="images/sdt1.png" align="center" width="85%" height="85%"/>
        * Steps:
            * Various forms ldr r0, [r1, r2] | ldr r0, [r1, r2, lsl #0]
            * Calculate the Target address
                * `Imm : Target = rn +/- offset  |  Reg : Traget = rn + shifted(rm) `
            * Load:
                * Either word | Byte
                * `Word : asp->regs[rd] = *((uint32_t*)Target)  |  Byte : asp->regs[rd] = (uint32_t)(*(uint8_t*))Target`, thus for Byte we want just a 8 bit value and not 32!!!
        * Understanding **str**:
            * ` Word : *((uint32_t*)Target) = asp->regs[rd] | Byte :*((uint8_t*))Target = (uint8_t)asp->regs[rd]`
        * OFFSET:
            * <img src="images/offset1.png" align="center" width="50%" height="50%"/>
            * <img src="images/offset2.png" align="center" width="50%" height="50%"/>
            * Extract the shift amount, identify the shift and do the shift.
            
   * Branches:
       * <img src="images/branch1.png" align="center" width="50%" height="50%"/>
       * <img src="images/branch2.png" align="center" width="50%" height="50%"/>
       * PC + 8, adding 8 as branch state will be 2 steps ahead of the actual branching.
   * Branch and Link:
       * If `link bit == 1`, then `lr = PC + 4`
   * `cmp/ bcc` (bcc : beq, bne, blt etc)
       * CPSR bits(Current Program Status Register):
           * N(-ve), Z(0), C(carry), V(overflow)
           * How to compute NZCV bits ?
               * cmp r0, r1 : works as signed bits i.e. r = r0 - r1
               * <img src="images/cmp1.png" align="center" width="50%" height="50%"/>
               * When doing C, we are doing unsigned value, i.e. if r1 is larger than r0 then C = 1. ((unsigned)r > 1)
               * **Understanding Overflow, if we add 2 large +ve numbers than the result might be outside the 32 bit value, or vice-versa for the -ve numbers.**
               * <img src="images/cmp2.png" align="center" width="50%" height="50%"/>
               * How to decide the cond ? **Referring to the condition code**
               * <img src="images/cmp3.png" align="center" width="75%" height="75%"/>

[22/03/2022]
1. Dynamic Analysis and Cache Simulation:
    - Adv(using emulator) : we have full visibility of what's happening under the hood.
    - <img src="images/dynamic1.png" align="center" width="25%" height="25%"/>, refer project 04 src page to get the list of items to be included.
2. Cache Memories:
    - requestor(need data) -> cache(small but fast) -> large dataset(large but slow)
    - <img src="images/cache1.png" align="center" width="50%" height="50%"/>
    - Replacement/ eviction Policy
        - Write policy (write back, write through(writing both in the cache and the larger mem))
    - Unix cmds : >free.
    - <img src="images/cache2.png" align="center" width="50%" height="50%"/>
    - Principle of Locality
        - Temporal : Accessing a fn call in a loop.
        - Spatial : Accessing next instruction in a code.
    - Caches and Address:
        * Direct Mapped Cached (based on address)
        * <img src="images/directMap1.png" align="center" width="25%" height="25%"/>, lower 2 bits would be the offset.
        * Imp, <img src="images/directCache2.png" align="center" width="50%" height="50%"/>
        * **valid bit** : Should be initially 0 and for a hit it should be set to '1'.(Initially when the system is booted there is no valid data in the cache)